# Function Registry

_Register functions to enable chat models to call them._

The hidden power behind `chat.register(ƒ)` is the Function Registry. It lets you register _typed_ functions quickly and easily so they can be called on behalf of chat models.


In [2]:
from chatlab import FunctionRegistry


def f(x: float):
    '''Multiply x by 2.'''
    return x * 2


registry = FunctionRegistry()
registry.register_function(f)

await registry.call("f", "{\"x\": 4}")

8

## Registering via Decorators

`register` is also a decorator, allowing you to register a function while declaring it.


In [13]:
from typing import Tuple, Optional
import math


@registry.register
def quadratic_formula(a: float, b: float, c: float) -> Optional[Tuple[float, float]]:
    """
    Solves the quadratic equation ax^2 + bx + c = 0.
    :param a: Coefficient of x^2
    :param b: Coefficient of x
    :param c: Constant term
    :return: Tuple containing two solutions or None if no real solutions exist
    """
    discriminant = b**2 - 4 * a * c
    if discriminant < 0:
        return None  # No real solutions
    x1 = (-b + math.sqrt(discriminant)) / (2 * a)
    x2 = (-b - math.sqrt(discriminant)) / (2 * a)
    return x1, x2


await registry.call('quadratic_formula', '{"a": 1, "b": 0, "c": -1}')

(1.0, -1.0)

Functions can be registered by:

- Passing a typed function in directly
- Passing the function and the Pydantic model that describes its parameters
- Passing the function and a JSON schema that describes its parameters

## Registering Typed Functions

> [!NOTE]
> Not all function parameters can be accurately determined. For those cases it's best to use a Pydantic model or JSON schema.
>
> If you're adventurous, make a [PR to chatlab](https://github.com/rgbkrk/chatlab) to improve the type inference!

### No Parameters


In [4]:
import random


def flip_a_coin():
    '''Flip a coin.'''
    return random.choice(["heads", "tails"])


registry.register(flip_a_coin)

{'name': 'flip_a_coin',
 'description': 'Flip a coin.',
 'parameters': {'type': 'object', 'properties': {}, 'required': []}}

### Multiple Parameters


In [5]:
def add(x: float, y: float):
    '''Add x and y.'''
    return x + y


registry.register(add)

{'name': 'add',
 'description': 'Add x and y.',
 'parameters': {'type': 'object',
  'properties': {'x': {'type': 'number'}, 'y': {'type': 'number'}},
  'required': ['x', 'y']}}

### Optional Parameters


In [8]:
def add(x: float, y: float = 1):
    '''Add x and y.'''
    return x + y


registry.register(add)

{'name': 'add',
 'description': 'Add x and y.',
 'parameters': {'type': 'object',
  'properties': {'x': {'type': 'number'}, 'y': {'type': 'number'}},
  'required': ['x']}}

## Registering Functions with Pydantic Models

Pydantic Models give you all the benefits above _and_ much more:

- Default values
- Descriptions on parameters
- Deeper type validation


In [19]:
from chatlab import FunctionRegistry
from pydantic import BaseModel

registry = FunctionRegistry()


class CompoundInterestModel(BaseModel):
    principal: float
    rate: float
    times_compounded: int
    years: float


@registry.register(parameter_schema=CompoundInterestModel)
def compound_interest(principal: float, rate: float, times_compounded: int, years: float) -> float:
    """
    Calculates the future value of an investment using compound interest.
    :param principal: Initial investment amount
    :param rate: Annual interest rate (as a decimal)
    :param times_compounded: Number of times interest is compounded per year
    :param years: Number of years the money is invested
    :return: Future value of the investment
    """
    return principal * (1 + rate / times_compounded) ** (times_compounded * years)


await registry.call(
    "compound_interest", CompoundInterestModel(principal=1000, rate=0.05, times_compounded=12, years=5).json()
)

1283.3586785035118

In [5]:
from chatlab import FunctionRegistry

registry = FunctionRegistry()


@registry.register
def ok():
    """fine"""
    return "ok"


await registry.call('ok')

'ok'

In [8]:
def also_fine():
    """This function is fine"""
    return "so fine"


registry.register(also_fine)

{'name': 'also_fine',
 'description': 'This function is fine',
 'parameters': {'type': 'object', 'properties': {}, 'required': []}}

In [9]:
await registry.call("also_fine")

'so fine'